In [13]:
%run functions.ipynb

Ready: 2023-04-15 11:11:47


In [4]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

# Новосибирск

In [143]:
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\_Final\Мастер_таблица_2ГИС_Новосибирск_230322_2345.xlsx")[['Рубрика']].iloc[:70]
name = 'novosib'

In [ ]:
make_dates_file(name)

In [ ]:
make_linear_plots('files/novosib_dates.xlsx', rubrics, cnt=25)

In [25]:
copy_images_texts('files/novosib_dates.xlsx', r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Новосибирск\Статья_Новосибирский распорядок_230401_1430.docx", True)

files/novosib_dates_report_230402_2213.docx


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  2.97it/s]


# Москва

In [4]:
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Москва\Мастер_таблица_2ГИС_Москва_230320_0245.xlsx")[['Рубрика']].iloc[:70]
name = 'moscow'

In [10]:
make_dates_file(name)

In [ ]:
make_linear_plots(f'files/df_dates_{name}.xlsx', rubrics, cnt=25)

In [4]:
#files/df_dates_moscow_report_230409_1840.docx

In [19]:
spark.stop()

# Алматы

In [7]:
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Алматы\Мастер_таблица_2ГИС_Алматы_230326_0030.xlsx")[['Рубрика']].iloc[:70]
name = 'almati'

In [10]:
make_dates_file(name)

In [ ]:
make_linear_plots(f'files/df_dates_{name}.xlsx', rubrics, cnt=25)

In [12]:
#files/df_dates_almati_report_230409_1910.docx

# Астана

In [24]:
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Астана\Мастер_таблица_2ГИС_Астана_230409_1645.xlsx").sort_values('Запросы, итого', ascending=False)[['Рубрика']].iloc[:70]
name = 'astana'

In [26]:
make_dates_file(name)

In [ ]:
make_linear_plots(f'files/df_dates_{name}.xlsx', rubrics, cnt=25)

In [12]:
#files/df_dates_astana_report_230409_1937.docx

# Калининград

In [6]:
name = 'kaliningrad'
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Калининград\Мастер_таблица_2ГИС_Калининград_230412_2345.xlsx").sort_values('Запросы, итого',ascending=False)[['Рубрика']].iloc[:70]

In [7]:
make_dates_file(name)

In [ ]:
#files/df_dates_kaliningrad_report_230415_1052.docx

In [ ]:
make_linear_plots(f'files/df_dates_{name}.xlsx', rubrics, cnt=25)

# 10 Городов

In [3]:
df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', IntegerType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
df_list = [i for i in files_list_chain if 'Астана' in i or 'Бишкек' in i or 'Москва' in i or 'ОАЭ' in i or 'Ташкент' in i]

for file in tqdm(df_list):
    df = spark.read.option("header","true").csv(file, schema=df_schema)
    df.createOrReplaceTempView('df')
    city = file.split('\\')[2]
    df_new = spark.sql(f'''
    select 
    rubricName
    , "{city}" city
    , time
    , year(time) year_r
    , hour(time) hour_r
    , SUBSTR(time, 6, 5) date_r
    , case when weekofyear(time) > 9 then 1 else weekofyear(time) end week_r
    , case when extract(dayofweek from time) = 1 then 7
        else extract(dayofweek from time)-1
    end week_day_r

    from df
    ''').where('year_r in (2020,2021,2022,2023) and month(time) < 3 ')

    df_new.write.mode('append').parquet(f'files/ten_city/df_dates')

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [53:00<00:00, 159.03s/it]


In [27]:
def count_dates(col, df_new='df_new'):
    dff1 = spark.sql(f'''

    with t as
        (
        select 
            year_r
            , rubricName
            , city
            , {col}
            , count(*) cnt
        from {df_new}
        group by
            year_r
            , rubricName
            , city
            , {col}
        order by
            year_r
            , rubricName
            , city
            , {col}
        )
    ,gr as
        (
        select
            rubricName 
            ,{col} 
            ,max(case when city = 'Астана'  then cnt end) cnt_a
            ,max(case when city = 'Бишкек'  then cnt end) cnt_b
            ,max(case when city = 'Москва'  then cnt end) cnt_m
            ,max(case when city = 'ОАЭ'     then cnt end) cnt_o
            ,max(case when city = 'Ташкент' then cnt end) cnt_t
        from t
        group by
            rubricName
            ,{col}
        order by
            rubricName
            ,{col}    
        )

    select
        rubricName 
        ,{col}
        ,sum(cnt_a) over(partition by rubricName, {col}) / sum(cnt_a) over(partition by rubricName)*100 cnt_a
        ,sum(cnt_b) over(partition by rubricName, {col}) / sum(cnt_b) over(partition by rubricName)*100 cnt_b
        ,sum(cnt_m) over(partition by rubricName, {col}) / sum(cnt_m) over(partition by rubricName)*100 cnt_m
        ,sum(cnt_o) over(partition by rubricName, {col}) / sum(cnt_o) over(partition by rubricName)*100 cnt_o
        ,sum(cnt_t) over(partition by rubricName, {col}) / sum(cnt_t) over(partition by rubricName)*100 cnt_t
        
        ,case when row_number() over(partition by rubricName order by cnt_a desc) < 4 then row_number() over(partition by rubricName order by cnt_a desc) end max_a
        ,case when row_number() over(partition by rubricName order by cnt_b desc) < 4 then row_number() over(partition by rubricName order by cnt_b desc) end max_b
        ,case when row_number() over(partition by rubricName order by cnt_m desc) < 4 then row_number() over(partition by rubricName order by cnt_m desc) end max_m
        ,case when row_number() over(partition by rubricName order by cnt_o desc) < 4 then row_number() over(partition by rubricName order by cnt_o desc) end max_o
        ,case when row_number() over(partition by rubricName order by cnt_t desc) < 4 then row_number() over(partition by rubricName order by cnt_t desc) end max_t
        
        ,case when row_number() over(partition by rubricName order by cnt_a) < 4 then row_number() over(partition by rubricName order by cnt_a) end min_a
        ,case when row_number() over(partition by rubricName order by cnt_b) < 4 then row_number() over(partition by rubricName order by cnt_b) end min_b
        ,case when row_number() over(partition by rubricName order by cnt_m) < 4 then row_number() over(partition by rubricName order by cnt_m) end min_m
        ,case when row_number() over(partition by rubricName order by cnt_o) < 4 then row_number() over(partition by rubricName order by cnt_o) end min_o
        ,case when row_number() over(partition by rubricName order by cnt_t) < 4 then row_number() over(partition by rubricName order by cnt_t) end min_t

    from gr

    ''').toPandas()
    return dff1

def make_linear_plots(input_filename, rubrics_df, cnt=9999, size_x=17.5, size_y=10.8, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    def zero(x,y):
        if x < 0:
            return str(0) +'%'
        return str(round(x,1)) +'%'
    
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    word_file = f'{input_filename[:-5]}_report_{dt_now}.docx'
    print(word_file)
    cols1 = ['Рубрика','Час','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    cols2 = ['Рубрика','Дата','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    cols3 = ['Рубрика','Неделя','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    cols4 = ['Рубрика','День недели','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    dff1 = pd.read_excel(input_filename, sheet_name='Часы')[cols1]
    dff2 = pd.read_excel(input_filename, sheet_name='Даты')[cols2]
    dff3 = pd.read_excel(input_filename, sheet_name='Недели')[cols3]
    dff4 = pd.read_excel(input_filename, sheet_name='Дни недели')[cols4]
    
    i = 0
    for rubric in tqdm(rubrics_df[:].iloc[:,0].values[0:cnt]):
        dff1_test = dff1[dff1['Рубрика'] == rubric].set_index(dff1.columns[1])
        dff2_test = dff2[dff2['Рубрика'] == rubric].set_index(dff2.columns[1])
        dff3_test = dff3[dff3['Рубрика'] == rubric].set_index(dff3.columns[1])
        dff4_test = dff4[dff4['Рубрика'] == rubric].set_index(dff4.columns[1])
        dff4_test.index = (['пн','вт','ср','чт','пт','сб','вс'])
        dff1_test.index = dff1_test.index.astype(str)
        i+=1
        j=0
        for d in [dff1_test, dff2_test, dff3_test, dff4_test]: 
            j+=1
            fig, ax = plt.subplots()
            ax = sns.lineplot(data = d, linewidth=3, palette=['#4F81BD', '#F79646', '#C0504D', '#9BBB59', '#264478'])
            plt.rcParams['figure.figsize']=17,10.8
            plt.rcParams['font.size'] = '16'
            plt.rcParams['figure.dpi'] = 2**8
            plt.rcParams['figure.facecolor'] = 'white'
            fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
            
            
            plt.legend(['Астана', 'Бишкек', 'Москва', 'ОАЭ', 'Ташкент'], prop=fpath)
            plt.xticks(font=fpath)
            plt.yticks(font=fpath)
            plt.grid(visible=True, linewidth=0.3)
            ax.set_xlabel('')
            ax.yaxis.set_major_formatter(FuncFormatter(zero))
            ax.xaxis.set_major_locator(ticker.MultipleLocator(1)) 
            ax.yaxis.set_major_locator(ticker.LinearLocator(numticks=6))  
            title_text = f'{i:02}/{j}. {rubric}. '
            img_file_name = input_filename.replace('files/','')[:-5]
            img_file_name_final = f'pictures/{img_file_name}_{i:02}_{j}.png'
            if 'Час' == d.index.name:
                ax.set_title(f'{title_text}Запросы по часам, шт.', font=fpath)
            elif 'Дата' == d.index.name:
                ax.xaxis.set_major_locator(ticker.MultipleLocator(4)) 
                ax.set_title(f'{title_text}Запросы по датам, шт.', font=fpath)
            elif 'Неделя' == d.index.name:
                ax.set_title(f'{title_text}Запросы по неделям, шт.', font=fpath)
            else:
                ax.set_title(f'{title_text}Запросы по дням недели, шт.', font=fpath) 
            
            im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354.png")
            ax.figure.figimage(im, 3250, 150, zorder=1, alpha=0.5)
          
            im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354.png")
            ax.figure.figimage(im, 200, 1800, zorder=1, alpha=0.5)

            ax.figure.savefig(img_file_name_final, bbox_inches='tight')
            add_image(word_file, img_file_name_final, rubric, img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
            #plt.show()
            plt.close()

In [9]:
df_new = spark.read.parquet(f'files/ten_city/df_dates')
df_new.createOrReplaceTempView('df_new')

In [28]:
df_new_list = []

In [29]:
for i in tqdm(['hour_r','date_r','week_r','week_day_r']):
    df_new_list.append(count_dates(i))

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [12:04<00:00, 181.12s/it]


In [30]:
dff1 = df_new_list[0]
dff2 = df_new_list[1]
dff3 = df_new_list[2]
dff4 = df_new_list[3]

In [34]:
t = ['Рубрика', 'Час','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент',
    'max_a', 'max_b', 'max_m', 'max_o', 'max_t', 'min_a', 'min_b', 'min_m', 'min_o', 'min_t']


t[1] = 'Час'
dff1.columns = t
t[1] = 'Дата'
dff2.columns = t
t[1] = 'Неделя'
dff3.columns = t
t[1] = 'День недели'
dff4.columns = t

with pd.ExcelWriter(f'files/ten_city/df_dates.xlsx') as writer:
    dff1.to_excel(writer, sheet_name='Часы', index=False)
    dff2.to_excel(writer, sheet_name='Даты', index=False)
    dff3.to_excel(writer, sheet_name='Недели', index=False)
    dff4.to_excel(writer, sheet_name='Дни недели', index=False)

In [41]:
df_tx_grouped_filtered = spark.read.parquet(f"files/ten_city/df_tx_grouped_filtered_ten_city")
df_tx_grouped_filtered.createOrReplaceTempView('df_tx_grouped_filtered')
rubrics = spark.sql('''select distinct rubricName from df_tx_grouped_filtered''').collect()
rubrics = [i.asDict()['rubricName'] for i in rubrics]
rubrics = pd.DataFrame(rubrics)
rubrics['Рейтинг рубрики'] = rubrics.index + 1

In [75]:
make_linear_plots(f'files/ten_city/df_dates.xlsx', rubrics, 25)

files/ten_city/df_dates_report_230422_0121.docx


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [06:11<00:00, 14.87s/it]
